In [ ]:
ans_file=""
text=""
question=""

In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import re
import time
import papermill as pm
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Loading Model and Encoder
model=torch.load("MODEL_APR_23")
model.eval()
tokenizer = torch.load("tokenizer_encoder.unknown")

In [3]:
def question_answer_A(question, text):
    
    #tokenize question and text in ids as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    #number of tokens in segment A - question
    num_seg_a = sep_idx+1

    #number of tokens in segment B - text
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    answer = 'unable to find answer'
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    return torch.max(output.start_logits), answer.capitalize() #score ans

In [4]:
def chunk_text(text, max_len):
    # split the text into sentences
    sentences = re.findall('[^.!?]+[.!?]', text)
    
    # initialize variables
    current_len = 0
    current_chunk = []
    chunks = []
    
    # iterate over each sentence
    for sentence in sentences:
        # add the sentence to the current chunk
        current_chunk.append(sentence)
        
        # update the length of the current chunk
        current_len += len(sentence)
        
        # if the length of the current chunk is greater than or equal to the max length
        if current_len >= max_len:
            # add the current chunk to the list of chunks
            chunks.append(' '.join(current_chunk))
            
            # reset the current chunk and length
            current_chunk = []
            current_len = 0
    
    # add any remaining sentences as a final chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

In [5]:
def get_better_ans(question, text):
  chunk_of_text=chunk_text(text,450)
  scores = {}
  for i in chunk_of_text:
    scores.update({question_answer_A(question, i)[0] : question_answer_A(question, i)[1] })
  v = list(scores.values())
  k = list(scores.keys())
  return (v[k.index(max(k))])

In [9]:
print(text)
print(question)
x = get_better_ans(question, text)


In [10]:
print(x)
f = open(ans_file, "w")
f.write(x)
f.close()

Aircraft
